In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
### Data cleaning

# Import data, add target classification, merge dataset, add label
fake = pd.read_csv("data/fake.csv", usecols= ["text"])
true = pd.read_csv("data/true.csv", usecols= ["text"])
fake['target'] = 0
true['target'] = 1
data = true.append(fake).sample(frac=1).reset_index().drop(columns=['index'])


In [13]:
data

,text,target
0,BERLIN (Reuters) - Foreign Minister Sigmar Gab...,1
1,"Make no mistake, the Muslim European invaders ...",0
2,WASHINGTON (Reuters) - The head of a hard-line...,1
3,Seven Delaware state workers were indicted and...,0
4,North Carolina Republicans just would not list...,0
...,...,...
44893,Rep. Trey Gowdy was full of sarcasm (rightfull...,0
44894,The Senate had better stick to their guns on t...,0
44895,WASHINGTON (Reuters) - President Donald Trump ...,1
44896,BALTIMORE (Reuters) - A Maryland state senator...,1
